In [1]:
#### COLAB ONLY CELL
from google.colab import drive
drive.mount('/content/gdrive')
# drive.mount('/content/gdrive',force_remount=True)
%cd /content/gdrive/MyDrive/COMEXT
%pip install -q -r ./comext_graph_analysis/requirements.txt
import hashlib, random, math, os, sys, pickle, time, zipfile, io, json#, powerlaw #,eurostat
import pandas as pd
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm, trange
from networkx.classes.function import degree
import matplotlib.pyplot as plt
from matplotlib import cm

sys.path.append('/content/gdrive/MyDrive/COMEXT/comext_graph_analysis/src')
import graph_analysis as g

Mounted at /content/gdrive
/content/gdrive/MyDrive/COMEXT


In [ ]:
# This script is to run the SBM on all years, all products graphs
# !Rscript world-trade-thesis/src/complete_sbm.r

In [147]:
dfs_prod = {}
for f in tqdm(os.listdir("data-samples/sbm")):
    df = pd.read_csv("data-samples/sbm/"+f,na_filter=False)
    f_l = f.split("_")
    y = f_l[1][1:]
    p = f_l[2][1:3]
    # try:
    if p not in dfs_prod:
        dfs_prod[p] = df.set_index("country").rename({"block":y},axis=1)
    else:
        dfs_prod[p] = pd.concat([dfs_prod[p], df.set_index("country").rename({"block":y},axis=1)],axis=1)

  0%|          | 0/1079 [00:00<?, ?it/s]

In [148]:
for p in tqdm(dfs_prod):
    dfs_prod[p].to_csv(f"data-samples/sbm_allyears/complete_allyears_p{p}.csv")

  0%|          | 0/57 [00:00<?, ?it/s]

In [57]:
metrics = pd.read_parquet("data-samples/metrics/metrics_complete_V.parquet")

In [109]:
m2 = metrics.set_index(["country","prod","year"])
m2["sbm_block"] = 0
sbm = dfs_prod["30"].reset_index().rename({"30":"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")

In [157]:
metrics = metrics.sort_values(["country","prod","year"]).set_index(["country","prod","year"])

In [149]:
df_prods = None
for p in tqdm(dfs_prod):
    if df_prods is None:
        sbm = dfs_prod[p]
        sbm[p] = p
        df_prods = sbm.reset_index().rename({p:"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")
    else:
        sbm = dfs_prod[p]
        sbm[p] = p
        df_prods = pd.concat([df_prods,sbm.reset_index().rename({p:"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")])sbms

  0%|          | 0/57 [00:00<?, ?it/s]

country  prod      
AD       30    2002    13.0
               2003     1.0
               2004     1.0
               2005     1.0
               2006     1.0
                       ... 
TT       UU    2019     1.0
               2020     1.0
LT       UU    2020     3.0
NE       UU    2020     1.0
SO       UU    2020     1.0
Name: sbm_block, Length: 196846, dtype: float64

In [158]:
sbms = df_prods.reset_index().rename({"level_2":"year"},axis=1).sort_values(["country","prod","year"]).reset_index(drop=True).set_index(["country","prod","year"]).values
metrics["sbm_block"] = sbms

In [161]:
metrics = metrics.reset_index()

In [163]:
metrics.to_parquet("data-samples/metrics/metrics_complete_V_sbm.parquet")